# Предсказание цены автомобиля

Необходимо обучить модель для определения рыночной стоимости автомобиля. В нашем распоряжении следующие данные: технические характеристики, комплектации и цены автомобилей.

## Оглавление:
* [1. Подготовка данных](#1)
* [2. Обучение моделей](#2)
* [3. Анализ моделей](#3)

# 1. Подготовка данных <a class="anchor" id="1"></a>

Импортируем библиотеки:

In [1]:
# <импорт библиотеки pandas>
import pandas as pd

# <импорт библиотеки sklearn>
import sklearn

# <Отключение предупреждений>
import warnings
warnings.filterwarnings('ignore')

# <импорт библиотеки numpy>
import numpy as np

Прочитаем файл с данными:

In [2]:
# <чтение файла с данными с сохранением в переменную df>
df = pd.read_csv('/datasets/autos.csv')

Рассмотрим информацию по датафрейму и первые 5 строк:

In [3]:
# <рассмотрим датафрейм df_region_0>
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB
None


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Подробнее опишем значение каждого атрибута.

Признаки

* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

Целевой признак

* Price — цена (евро)

Создадим отдельный датафрейм, который будем очищать от пропусков.

In [4]:
df_preprocessed=df

Сбросим столбцы дат, они не пригодятся для обучения.

In [5]:
# <сбросим три столбца, которые не помогут нам предсказать целевой признак>
df_preprocessed = df_preprocessed.drop(['DateCrawled','DateCreated','LastSeen'], axis=1)

Рассмотрим столбец *Power*:

In [6]:
df_preprocessed['Power'].describe()

count    354369.000000
mean        110.094337
std         189.850405
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

Определим сколько значений столбца *Power* аномальны.

In [7]:
print(df_preprocessed['Power'].quantile(0.11))
print(df_preprocessed['Power'].quantile(0.115))
print(df_preprocessed['Power'].quantile(0.12))

0.0
25.0
43.0


Заменим эти значения на медиану.

In [8]:
power_median = df_preprocessed[df_preprocessed['Power']>25]['Power'].median()

In [9]:
df_preprocessed['Power'] = (np.where((df_preprocessed.Power <= 25), 
                               power_median, 
                               df_preprocessed.Power))

Заполним пропуски в столбце *Model*, опираться будем на бренд и лошадиные силы. Заполним модой - наиболее часто встречающимся значением для определенных группы бренда и кол-ва лошадних сил. Если значения модели нет, будем проставлено значение unknown.

In [10]:
df_preprocessed['Model'] = (df_preprocessed.groupby(['Brand','Power'])['Model'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Заполним пропуски в столбце *VehicleType*, опираться будем на бренд и модель. Тут мы можем рассчитывать только на заполнение пропусков по моде, так как по имеющимся признакам предположить какой у машины тип кузова мы не в силах.

In [11]:
df_preprocessed['VehicleType'] = (df_preprocessed.groupby(['Brand','Model'])['VehicleType'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Заполним пропуски в столбце *FuelType*, опираться будем на бренд и модель. Стиль заполнения пропусков тот же.

In [12]:
df_preprocessed['FuelType'] = (df_preprocessed.groupby(['Brand','Model'])['FuelType'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Заполним пропуски в столбце *Gearbox*, опираться будем на бренд и модель. Стиль заполнения пропусков тот же.

In [13]:
df_preprocessed['Gearbox'] = (df_preprocessed.groupby(['Brand','Model'])['Gearbox'].transform(lambda x: x.fillna((x.mode()[0] if not x.mode().empty else "unknown"))))

Проверим гипотезу:
* Средняя цена у автомобилей с признаком *NotRepaired* = *yes* и *NotRepaired* = *nan* различается.

Сформулируем гипотезы:

**H₀:** Средняя цена у автомобилей с признаком *NotRepaired* = *yes* и *NotRepaired* = *nan* равна.

**H₁:** Средняя цена у автомобилей с признаком *NotRepaired* = *yes* и *NotRepaired* = *nan* различается.

In [14]:
# <Посчитаем среднее для обоих выборок>
yes_mean = df_preprocessed[df_preprocessed['NotRepaired'] == 'yes']['Price'].mean()
nan_mean = df_preprocessed[(df_preprocessed['NotRepaired'] != 'no') & (df_preprocessed['NotRepaired'] != 'yes')]['Price'].mean()
print('Средняя цена отремонтированного автомобиля', yes_mean)
print('Средняя ценам автомобиля, данные по ремонту которого неизвестны:', nan_mean)

Средняя цена отремонтированного автомобиля 1916.0390802684863
Средняя ценам автомобиля, данные по ремонту которого неизвестны: 2626.4192455800094


In [15]:
# <Создадим переменные для наших выборок>
yes_data = df_preprocessed[df_preprocessed['NotRepaired'] == 'yes']['Price']
nan_data = df_preprocessed[(df_preprocessed['NotRepaired'] != 'no') & (df_preprocessed['NotRepaired'] != 'yes')]['Price']

In [16]:
# <импорт библиотеки scipy>
from scipy import stats as st

In [17]:
# <Уровень значимости>
alpha = 0.05

# <Метод библиотеки scipy, позволяющий проверить гипотезу о равенстве двух средних>
results =  st.ttest_ind(
    yes_data, 
    nan_data)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 6.76283705089867e-266
Отвергаем нулевую гипотезу


Такая же ситуация ждет нас и с категорией *no*. Поэтому предлагаю заменить все *nan* значения на третью категорию *unknown*.

In [18]:
df_preprocessed['NotRepaired'] = (np.where(((df_preprocessed['NotRepaired'] != 'no') & (df_preprocessed['NotRepaired'] != 'yes')), 
                               'unknown', 
                               df_preprocessed.NotRepaired))

Рассмотрим столбец *RegistrationYear*:

In [19]:
df_preprocessed['RegistrationYear'].describe()

count    354369.000000
mean       2004.234448
std          90.227958
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

На сайте *auto.ru* наиболее старый год для поиска объявления - 1890. Сделаем тут также, заменим все аномалии модой.

In [20]:
df_preprocessed['RegistrationYear'] = (np.where(((df_preprocessed['RegistrationYear'] < 1890) | (df_preprocessed['RegistrationYear'] > 2020)), 
                               df_preprocessed['RegistrationYear'].mode(), 
                               df_preprocessed.RegistrationYear))

Рассмотрим столбец *Kilometer*:

In [21]:
df_preprocessed['Kilometer'].describe()

count    354369.000000
mean     128211.172535
std       37905.341530
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64

В нем все довольно адекватно.

Рассмотрим столбец *RegistrationMonth*:

In [22]:
df_preprocessed['RegistrationMonth'].value_counts()

0     37352
3     34373
6     31508
4     29270
5     29153
7     27213
10    26099
12    24289
11    24186
9     23813
1     23219
8     22627
2     21267
Name: RegistrationMonth, dtype: int64

Тут у нас 13 месяцев, но мы можем условиться, что 0 месяц - неизвестный, незаполненный. В условии задачи все равно не было сказано с какого числа считаются месяцы. Также заметим что скорее всего этот показатель будет иметь маленький вес.

Рассмотрим столбец *NumberOfPictures*:

In [23]:
df_preprocessed['NumberOfPictures'].value_counts()

0    354369
Name: NumberOfPictures, dtype: int64

Удалим этот столбец, он никак не будет влиять на обучение, если данные по нему у каждой строки одинаковые. Скорее всего такие значения результат ошибки загрзуки данных или хранения данных. Также удалим столбец *PostalCode*, почтовый индекс клиента анкеты никак не влияет на цену автомобиля.

In [24]:
# <сбросим столбец>
df_preprocessed = df_preprocessed.drop(['NumberOfPictures','PostalCode'], axis=1)

Проверим количество дубликатов:

In [25]:
# <проверим количество полных дубликатов>
df_preprocessed.duplicated().sum()

30187

In [26]:
df_preprocessed.duplicated().sum()/354369

0.08518521653982149

Это плата за мой способ заполнения пропусков. Мы потеряем около 6 % данных.

In [27]:
# <Сбросим дубликаты>
df_preprocessed = df_preprocessed.drop_duplicates().reset_index(drop = True)

### Вывод

Мы осмотрели данные и обработали пропуски. Данные практически подготовлены к обучению.

# 2. Обучение моделей <a class="anchor" id="2"></a>

Переведем тип данных категориальных столбцов в *category*:

In [28]:
df_preprocessed['VehicleType'] = df_preprocessed['VehicleType'].astype('category')
df_preprocessed['Gearbox'] = df_preprocessed['Gearbox'].astype('category')
df_preprocessed['Model'] = df_preprocessed['Model'].astype('category')
df_preprocessed['FuelType'] = df_preprocessed['FuelType'].astype('category')
df_preprocessed['Brand'] = df_preprocessed['Brand'].astype('category')
df_preprocessed['NotRepaired'] = df_preprocessed['NotRepaired'].astype('category')

Разделим датафрейм на признаки и целевой признак.

In [29]:
# <Разделим датафрейм на features и target - целевой признак>
target = df_preprocessed['Price']
features = df_preprocessed.drop('Price', axis=1)

Поделим датафрейм на обучающую и тестовую выборку.

In [30]:
# <Импортируем функцию из бибилиотеки sklearn>
from sklearn.model_selection import train_test_split

In [31]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state = 0)

Проведем масштабирование признаков:

In [32]:
# <импортируем StandardScaler из библиотеки sklearn>
from sklearn.preprocessing import StandardScaler

In [33]:
numeric = ['RegistrationYear','Power','Kilometer','RegistrationMonth']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Импортируем библиотеку *LightGBM*:

In [34]:
import lightgbm as lgb

Попробуем использовать эту библиотеку, начнем с дерева принятия решений с градиентным бустингом (*gbdt: traditional Gradient Boosting Decision Tree*).

Загружаем датасет в переменную, создаем словарь с параметрами:
* *boosting_type* - алгоритм градиентного бустинга. Именно тут мы выбрали дерево принятия решений с градиентным бустингом.
* *objective* - тут указывается регрессия это, либо классфикация.
* *metric* - метрика по которой будем оценивать качество модели.
* *learning_rate* - скорость обучения, при ее уменьшении можно получить более лучшие результаты.
* *num_iterations* - количество итераций обучения.
* *max_depth* - максимальная глубина дерева.
* *num_leaves*  - количество листьев дерева.

Время выполнения ячейки - 6 минут 8 секунд.

In [35]:
%%time
hyper_params = {
    'learning_rate': 0.003,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    "num_iterations": 2500,
    'max_depth': 10,
    'num_leaves': 100
}
gbdt = lgb.LGBMRegressor(**hyper_params)
gbdt.fit(features_train, target_train,
        eval_set=[(features_train, target_train)],
        eval_metric='rmse',
        early_stopping_rounds=10, verbose = 100)

Training until validation scores don't improve for 10 rounds
[100]	training's rmse: 3653.96
[200]	training's rmse: 3051.97
[300]	training's rmse: 2636.81
[400]	training's rmse: 2354.12
[500]	training's rmse: 2159.79
[600]	training's rmse: 2027.77
[700]	training's rmse: 1938.29
[800]	training's rmse: 1876.4
[900]	training's rmse: 1832.99
[1000]	training's rmse: 1801.43
[1100]	training's rmse: 1776.75
[1200]	training's rmse: 1758.26
[1300]	training's rmse: 1743.31
[1400]	training's rmse: 1731.19
[1500]	training's rmse: 1721.28
[1600]	training's rmse: 1712.55
[1700]	training's rmse: 1704.73
[1800]	training's rmse: 1697.69
[1900]	training's rmse: 1691.13
[2000]	training's rmse: 1685.29
[2100]	training's rmse: 1679.55
[2200]	training's rmse: 1674.01
[2300]	training's rmse: 1669.34
[2400]	training's rmse: 1664.91
[2500]	training's rmse: 1660.67
Did not meet early stopping. Best iteration is:
[2500]	training's rmse: 1660.67
CPU times: user 6min 5s, sys: 2.36 s, total: 6min 8s
Wall time: 6min 

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.003, max_depth=10,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1,
              num_iterations=2500, num_leaves=100, objective='regression',
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
              task='train')

Теперь попробуем случайный лес. С ним обязательно нужно заполнить параметры *feature_fraction*, *bagging_fraction*, *bagging_freq*. Также увеличил количество листьев и кол-во деревьев.

Время выполнения ячейки - 5 минут 22 секунды.

In [37]:
%%time
hyper_params = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'regression',
    'metric': 'rmse',
    "num_iterations": 2500,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.3,
    'bagging_freq': 10,
    'num_leaves': 300,
    'max_depth': 8,
    'n_estimators': 200
    
}
rf = lgb.LGBMRegressor(**hyper_params)
rf.fit(features_train, target_train,
        eval_set=[(features_train, target_train)],
        eval_metric='rmse',
        early_stopping_rounds=10, verbose = 250)

Training until validation scores don't improve for 10 rounds
[250]	training's rmse: 2202.31
[500]	training's rmse: 2226.43
[750]	training's rmse: 2221.67
[1000]	training's rmse: 2220.54
[1250]	training's rmse: 2227.99
[1500]	training's rmse: 2231.32
[1750]	training's rmse: 2235.11
[2000]	training's rmse: 2233.7
[2250]	training's rmse: 2231.96
[2500]	training's rmse: 2232.83
Did not meet early stopping. Best iteration is:
[41]	training's rmse: 2103.92
CPU times: user 5min 20s, sys: 2.35 s, total: 5min 22s
Wall time: 5min 24s


LGBMRegressor(bagging_fraction=0.3, bagging_freq=10, boosting_type='rf',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
              importance_type='split', learning_rate=0.1, max_depth=8,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=200, n_jobs=-1,
              num_iterations=2500, num_leaves=300, objective='regression',
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
              task='train')

Теперь попробуем *Dropouts meet Multiple Additive Regression Trees*. Мною были изменены параметры скорости обучения и значения, которое определяет какое количество признаков использовать. Это необходимо чтобы обучение прошло быстрее, так как эта модель обучается медленно.

Время выполнения ячейки - 7 минут 28 секунд.

In [38]:
%%time
hyper_params = {
    'learning_rate': 0.7,
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': 'rmse',
    "num_iterations": 600,
    'feature_fraction': 0.5
}
dart_model = lgb.LGBMRegressor(**hyper_params)
dart_model.fit(features_train, target_train,
        eval_set=[(features_train, target_train)],
        eval_metric='rmse',
        early_stopping_rounds=10, verbose = 50)

[50]	training's rmse: 1777.73
[100]	training's rmse: 1706.85
[150]	training's rmse: 1663.86
[200]	training's rmse: 1633.84
[250]	training's rmse: 1616.81
[300]	training's rmse: 1598.64
[350]	training's rmse: 1589.55
[400]	training's rmse: 1572.88
[450]	training's rmse: 1557.22
[500]	training's rmse: 1543.29
[550]	training's rmse: 1532.05
[600]	training's rmse: 1530.61
CPU times: user 7min 28s, sys: 667 ms, total: 7min 28s
Wall time: 7min 32s


LGBMRegressor(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.5, importance_type='split', learning_rate=0.7,
              max_depth=-1, metric='rmse', min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_iterations=600, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, task='train')

Попробуем обучить обыкновенную линейную регрессию.

In [39]:
# <Преобразуем категориальные признаки в фиктивные переменные, и сбросим по одному из них у каждого признака.>
features_train_1 = pd.get_dummies(features_train, drop_first=True)

In [43]:
# <Импортируем метод логистической регрессии>
from sklearn.linear_model import LinearRegression
# <Импортируем функцию cross_val_score>
from sklearn.model_selection import cross_val_score

Время выполнения ячейки - 1 минута 50 секунд.

In [57]:
%%time
# <Создадим модель лог. регрессии,>
model_lr = LinearRegression()

# <Оценим качество модели, обученной в ходе перекрестной проверки>
score = cross_val_score(model_lr, features_train_1, target_train, cv=4, scoring='neg_mean_squared_error').mean()

# <обучаем модель> 
model_lr.fit(features_train_1, target_train)

CPU times: user 1min 27s, sys: 23 s, total: 1min 50s
Wall time: 1min 50s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Вывод

Мы обучили несколько моделей с градиентным бустингом, а также линейную регрессию. В следующем пункте проанализируем их скорость работы и качество.

# 3. Анализ моделей <a class="anchor" id="3"></a>

Проверим на тестовой выборке дерево принятия решений с градиентным бустингом.

In [66]:
from sklearn.metrics import mean_squared_error

In [67]:
predictions = gbdt.predict(features_test)
mean_squared_error(target_test,predictions)**0.5

1747.736487415676

Проверим на тестовой выборке случайный лес с градиентным бустингом.

In [68]:
predictions = rf.predict(features_test)
mean_squared_error(target_test,predictions)**0.5

2114.207575737013

Проверим на тестовой выборке *Dropouts meet Multiple Additive Regression Trees(DART)*.

In [69]:
predictions = dart_model.predict(features_test)
mean_squared_error(target_test,predictions)**0.5

1726.9859952149693

Проверим на тестовой выборке линейную регрессию.

In [70]:
# <Преобразуем категориальные признаки в фиктивные переменные, и сбросим по одному из них у каждого признака.>
features_test_1 = pd.get_dummies(features_test, drop_first=True)

In [71]:
predictions = model_lr.predict(features_test_1)
mean_squared_error(target_test,predictions)**0.5

3036.405060534749

Подытожим результаты:

In [77]:
results = {'model': ['Линейная регрессия', 'gbdt','rf','dart'], 
           'rmse': [3036.4, 1747.74, 2114.2, 1726.99],
           'time': ['1:50','6:08','5:22','7:28']
          }
results = pd.DataFrame(data=results)
results

,model,rmse,time
0,Линейная регрессия,3036.40,1:50
1,gbdt,1747.74,6:08
2,rf,2114.20,5:22
3,dart,1726.99,7:28


### Вывод

* Наиболее быстрой в плане обучения является модель линейной регресии. Однако если выставить определенное количество итераций три других метода за то же время что и линейная регрессия дадут лучший результат.
* Среди моделей с градиентным бустингом наиболее быстро обучилась модель случайного леса, но при этом на большинстве итераций не происходило положительной динамики улучшения *RMSE*
* Самой медленной моделью, но при этом выдающей самый качественный результат является модель *DART*, при определенных параметрах и большом количестве итераций можно было получить *RMSE* < 1000.
* Стабильной и в плане скорости, и в плане улучшения *RMSE* показала себя модель *gbdt*.